# Load tables

You can load data from PostgreSQL DB specifying credentials or just unpickle pandas dataframe from attached dataset

In [1]:
load_mode = 'unpickle'
# load_mode = 'sql'

In [2]:
!pip install psycopg2-binary

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/bc/0d/486e3fa27f39a00168abfcf14a3d8444f437f4b755cc34316da1124f293d/psycopg2_binary-2.9.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 30.5 MB/s eta 0:00:00


In [3]:
import sqlalchemy
import pandas as pd
from tqdm import tqdm

In [4]:
db_url = sqlalchemy.engine.URL.create(
    drivername='postgresql',
    username='***',
    password='***',
    host='***',
    port=1111,
    database='***')

engine = sqlalchemy.create_engine(db_url)

In [5]:
metadata_obj = sqlalchemy.MetaData()

if load_mode == 'sql':
    metadata_obj.reflect(bind=engine, only=['user_data', 'post_text_df', 'feed_data'])
    df_user = pd.read_sql(sqlalchemy.select(metadata_obj.tables['user_data']), engine)
    df_post = pd.read_sql(sqlalchemy.select(metadata_obj.tables['post_text_df']), engine)

In [6]:
def batch_load_sql(query, chunksize=200000) -> pd.DataFrame:
    engine = sqlalchemy.create_engine(db_url)
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in tqdm(pd.read_sql(query, conn, chunksize=chunksize)):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

In [7]:
if load_mode == 'sql':
    # 76892800 (68686455 if only 'view') rows / 200000 chunksize = 385 (344) iter
    df_feed = batch_load_sql(sqlalchemy
                             .select(metadata_obj.tables['feed_data'])
                             .where(metadata_obj.tables['feed_data'].c.action == 'view'))

### Unpickle

In [8]:
if load_mode == 'unpickle':
    df_feed = pd.read_pickle('/kaggle/input/kc-dl-prj3-catboost-dataset/df_feed.pkl')
    df_user = pd.read_pickle('/kaggle/input/kc-dl-prj3-catboost-dataset/df_user.pkl')
    df_post = pd.read_pickle('/kaggle/input/kc-dl-prj3-catboost-dataset/df_post.pkl')

In [9]:
df_feed.head(3)

,timestamp,user_id,post_id,action,target
0,2021-10-20 18:57:50,100224,2160,view,0
1,2021-10-20 19:00:22,100224,1760,view,0
2,2021-10-20 19:02:17,100224,684,view,0


In [10]:
df_user.head(3)

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads


In [11]:
df_post.head(3)

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business


# EDA

In [12]:
FEED_SIZE = 2_000_000
df_feed = df_feed.groupby('target').sample(FEED_SIZE//2)

In [13]:
df_feed['hour'] = df_feed.timestamp.dt.hour
df_feed['dayofweek'] = df_feed.timestamp.dt.dayofweek
df_feed['day'] = df_feed.timestamp.dt.day

In [14]:
df_feed = df_feed.merge(df_user, how='left', on='user_id').merge(df_post, how='left', on='post_id')

In [15]:
df_feed.head(3)

,timestamp,user_id,post_id,action,target,hour,dayofweek,day,gender,age,country,city,exp_group,os,source,text,topic
0,2021-12-25 11:10:55,161251,1829,view,0,11,5,25,0,19,Russia,Novosibirsk,4,iOS,organic,Collins named UK Athletics chief\n\nUK Athleti...,sport
1,2021-10-21 13:40:07,99104,3802,view,0,13,3,21,0,44,Ukraine,Pidhorodne,2,Android,ads,AHMEDABAD\nURGENT HELP NEEDED \n\nBLOOD TYPE :...,covid
2,2021-11-21 06:45:40,63562,5249,view,0,6,6,21,0,14,Russia,Opochka,2,Android,ads,"In 1850 in Yorkshire, a boy chimney-sweep is f...",movie


In [16]:
df_feed.timestamp.min(), df_feed.timestamp.max(), df_feed.timestamp.max()-df_feed.timestamp.min()

(Timestamp('2021-10-01 06:01:40'),
 Timestamp('2021-12-29 23:43:27'),
 Timedelta('89 days 17:41:47'))

In [17]:
split_point = df_feed[['timestamp']].quantile(0.8, numeric_only=False).squeeze()
split_point

Timestamp('2021-12-13 06:47:42.200000')

In [18]:
df_train = df_feed[df_feed.timestamp < split_point].drop(['timestamp', 'action'], axis=1)
df_test = df_feed[df_feed.timestamp >= split_point].drop(['timestamp', 'action'], axis=1)

In [19]:
df_train.target.value_counts()

target
0    815832
1    784168
Name: count, dtype: int64

In [20]:
df_test.target.value_counts()

target
1    215832
0    184168
Name: count, dtype: int64

In [21]:
num_cols = ['hour', 'dayofweek', 'day', 'age']
cat_cols = ['gender', 'country', 'exp_group', 'os', 'source', 'topic']
text_cols =  ['city', 'text']

In [22]:
df_train

,user_id,post_id,target,hour,dayofweek,day,gender,age,country,city,exp_group,os,source,text,topic
1,99104,3802,0,13,3,21,0,44,Ukraine,Pidhorodne,2,Android,ads,AHMEDABAD\nURGENT HELP NEEDED \n\nBLOOD TYPE :...,covid
2,63562,5249,0,6,6,21,0,14,Russia,Opochka,2,Android,ads,"In 1850 in Yorkshire, a boy chimney-sweep is f...",movie
3,137901,6736,0,14,0,8,0,24,Russia,Aleksandrov,4,Android,organic,"Personally, I find the movie to be quite a goo...",movie
4,135020,5272,0,14,2,20,1,39,Russia,Irkutsk,2,iOS,organic,"What can you say about a grainy, poorly filmed...",movie
5,158457,827,0,7,1,9,0,14,Russia,Tomsk,0,iOS,organic,Row threatens Hendrix museum plan\n\nProposals...,entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999993,151809,5217,1,7,3,2,0,26,Russia,Moscow,2,iOS,organic,This was awful. Andie Macdowell is a terrible ...,movie
1999994,21363,1531,1,7,2,10,1,19,Russia,Moscow,2,iOS,ads,Keegan hails comeback king Fowler\n\nMancheste...,sport
1999996,53790,1876,1,20,2,8,1,20,Russia,Moscow,2,iOS,ads,Laporte tinkers with team\n\nFrance coach Bern...,sport
1999998,94400,6839,1,7,4,8,1,51,Russia,Arsk,1,Android,ads,OK its not the best film Ive ever seen but at ...,movie


In [23]:
X_train = df_train.drop(['target', 'user_id', 'post_id'], axis=1)
X_test = df_test.drop(['target', 'user_id', 'post_id'], axis=1)

y_train = df_train['target']
y_test = df_test['target']

# Model

In [24]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
                           used_ram_limit='22gb',
                           iterations=200,
                           learning_rate=1,
                           depth=2)

model.fit(X_train,
          y_train,
          cat_features=cat_cols,
          text_features=text_cols,
          eval_set=(X_test,y_test),
          verbose=10,
          use_best_model=True)

0:	learn: 0.6822516	test: 0.6863945	best: 0.6863945 (0)	total: 3.11s	remaining: 10m 18s
10:	learn: 0.6719412	test: 0.6802253	best: 0.6791848 (4)	total: 26.7s	remaining: 7m 38s
20:	learn: 0.6697809	test: 0.6785159	best: 0.6784691 (19)	total: 49s	remaining: 6m 57s
30:	learn: 0.6685566	test: 0.6774916	best: 0.6774846 (29)	total: 1m 11s	remaining: 6m 30s
40:	learn: 0.6663139	test: 0.6755065	best: 0.6755065 (40)	total: 1m 33s	remaining: 6m 1s
50:	learn: 0.6652702	test: 0.6745258	best: 0.6745258 (50)	total: 1m 54s	remaining: 5m 33s
60:	learn: 0.6625722	test: 0.6724237	best: 0.6724237 (60)	total: 2m 17s	remaining: 5m 12s
70:	learn: 0.6612337	test: 0.6713301	best: 0.6713301 (70)	total: 2m 38s	remaining: 4m 48s
80:	learn: 0.6591739	test: 0.6695369	best: 0.6695369 (80)	total: 3m 1s	remaining: 4m 25s
90:	learn: 0.6583663	test: 0.6689582	best: 0.6689582 (90)	total: 3m 23s	remaining: 4m 3s
100:	learn: 0.6576228	test: 0.6684056	best: 0.6684056 (100)	total: 3m 43s	remaining: 3m 39s
110:	learn: 0.6571

CatBoost is using more CPU RAM (22.8GiB) than the limit (22GiB)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(130240512) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(130240512) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(130240512) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(130240512) > ResourceQuota(0)


In [25]:
from sklearn.metrics import roc_auc_score

print(f"Качество на трейне: {roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])}")
print(f"Качество на тесте: {roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])}")

Качество на трейне: 0.6549126151945693
Качество на тесте: 0.6400275423351892


In [26]:
model.save_model('catboost_model', format="cbm")              